In [2]:
import itertools
import pandas as pd
from collections import defaultdict
from itertools import islice, chain
import re
from datetime import datetime
from datetime import timedelta
import networkx as nx
import numpy as np
import os


In [14]:
data = pd.read_csv('company_relation.csv')
kg = data.drop(['Unnamed: 0'],axis =1)
kg = kg.drop(kg[kg['relation']=='unknown'].index)
kg

,head,tail,relation,sentence,time,head_code,tail_code
0,上海机场,白云机场,same_industry,继续推荐国际旅客占比高、免税经营更为灵活的上海机场，关注产能空间持续打开，商业消费有望迎高成...,2022-02-14 08:58:14,600009.SH,600004.SH
1,赣锋锂业,中铝国际,rise,有色金属板块集体上涨，赣锋锂业涨7%，中铝国际涨超4%,2022-02-17 12:08:27,002460.SZ,601068.SH
2,贵州茅台,宁德时代,compete,贵州茅台超过宁德时代，重回公募基金头号重仓股,2022-01-25 17:10:52,600519.SH,300750.SZ
5,常熟银行,江苏银行,rise,截至9:50，银行ETF华夏（515020）成分股常熟银行大涨超4%，江苏银行涨超3%,2022-01-13 10:08:37,601128.SH,600919.SH
6,比亚迪,广汽集团,rise,港股汽车股走高，理想汽车涨超4%，小鹏汽车涨超2%，比亚迪股份、广汽集团跟涨,2022-01-14 15:30:39,002594.SZ,601238.SH
...,...,...,...,...,...,...,...
8595,五矿稀土,中信证券,rise,五矿稀土三日内累涨23%，深股通买入10.58亿元并卖出10.44亿元，中信证券上海溧阳路买...,2021-08-09 16:43:00,000831.SZ,600030.SH
8596,东风汽车,亿华通,same_industry,公司客户有东风汽车集团、吉利汽车、柳州五菱、PLUG，POWER、亿华通、上海重塑、康明...,2021-08-09 16:15:00,600006.SH,688339
8598,洛阳钼业,中国铝业,rise,南方锰业涨13.7%，洛阳钼业涨6.5%，兴发铝业涨1.1%，中国铝业涨0.4%,2021-08-09 13:11:00,603993.SH,601600.SH
8599,星网锐捷,润和软件,cooperate,星网锐捷消息，近日，星网锐捷旗下升腾资讯与润和软件达成战略合作，发布全球首款基于OpenHa...,2021-08-09 13:06:00,002396.SZ,300339.SZ


In [36]:
import dateutil.parser

def getDateTimeFromISO8601String(s):
    d = dateutil.parser.parse(s)
    date = d.strftime('%Y%m%d')
    
    return date

In [31]:
def build_links(kg_df):    
    links = []
    ts = []
    for i in range(len(kg_df)):
        timesting = kg_df['time'][i]
        timestamp = getDateTimeFromISO8601String(timesting)
        ts.append(timestamp)
        links.append((kg_df['head_code'][i],kg_df['relation'][i],kg_df['tail_code'][i],timestamp))
    return links,ts

In [32]:
kg= kg.sort_values(['time'])
kg = kg.reset_index()

In [37]:
links,ts = build_links(kg)

In [22]:
relation_ls = kg['relation'].unique()

relation2id = {}
for i in range(len(relation_ls)):
    relation2id[relation_ls[i]] = i

In [ ]:
company2id = {}
count = 0
graph = []
for (a,b,c,time) in links:
    if a not in company2id:
        company2id[a] = count
        count += 1
    if c not in company2id:
        company2id[c] = count 
        count += 1

    trip = dict()
    trip['Node_one'] = company2id[a]
    trip['Node_two'] = company2id[c]
    trip['Edge_type'] = relation2id[b]
    trip['Timestamp'] = time

    if trip not in graph:
        graph.append(trip)

In [28]:
graph_d = pd.DataFrame(graph)
graph_d

,Node_one,Node_two,Edge_type,Timestamp
0,0,1,0,2021-08-09 04:01:05
1,2,3,0,2021-08-09 05:12:25
2,4,5,1,2021-08-09 06:06:15
3,6,7,2,2021-08-09 09:41:41
4,8,9,3,2021-08-09 09:45:00
...,...,...,...,...
3994,1438,1168,4,2022-02-24 18:21:36
3995,1426,6,4,2022-02-24 18:46:06
3996,1426,6,4,2022-02-24 18:47:00
3997,1426,6,4,2022-02-24 18:47:19


In [29]:
graph_d.to_csv('K-Q_train.txt', header=None, index=None, sep='\t', mode='a')

In [37]:
def build_subgraph(SLICE_WEEKS,links,ts,relation2id):
    
    START_DATE = min(ts)
    MAX_DATE = max(ts)
    print ("Start date", START_DATE)
    slice_id = 1

    company2id = {}
    count = 0
    graph = []

    # Split the set of links in order by slices to create the graphs.

    for (a, b, c,time) in links:
        
        prev_slice_id = slice_id

        datetime_object = time
        if datetime_object > MAX_DATE:
            months_diff = (MAX_DATE - START_DATE).days//7  # 
        else:
            months_diff = (datetime_object - START_DATE).days//7
        slice_id = months_diff // SLICE_WEEKS
        slice_id = max(slice_id, 1) 
        

        if a not in company2id:
            company2id[a] = count
            count += 1
        if c not in company2id:
            company2id[c] = count 
            count += 1
    
        trip = dict()
        trip['Node_one'] = company2id[a]
        trip['Node_two'] = company2id[c]
        trip['Edge_type'] = relation2id[b]
        trip['Timestamp'] = slice_id
        
        if trip not in graph:
            graph.append(trip)

    return graph,company2id    

In [38]:
SLICE_WEEKS = 2
graph,company2id = build_subgraph(SLICE_WEEKS,links,ts,relation2id)

Start date 2021-08-09 04:01:05


In [39]:
graph_d = pd.DataFrame(graph)

In [44]:
# split train, test, validation data 

graph_d[:2000]

,Node_one,Node_two,Edge_type,Timestamp
0,0,1,0,1
1,2,3,0,1
2,4,5,1,1
3,2,6,2,1
4,7,8,3,1
...,...,...,...,...
1995,636,8,6,7
1996,185,309,12,7
1997,7,8,0,7
1998,1078,65,9,7


In [43]:
graph_d.to_csv('K-quant_dynamicKG.txt', header=None, index=None, sep='\t', mode='a')